In [1]:
import tensorflow as tf 
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')
tf.test.is_built_with_cuda() 

ImportError: cannot import name 'saveable_objects_from_trackable' from 'tensorflow.python.training.saving.saveable_object_util' (C:\Users\CSAS\anaconda3\envs\gpu2\lib\site-packages\tensorflow\python\training\saving\saveable_object_util.py)

In [ ]:
device_lib.list_local_devices()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
from pytorch3d.utils import ico_sphere
from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import chamfer_distance

class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),  # normalize inputs to [-1, 1] so make outputs [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)


# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1  # 784
batch_size = 32
num_epochs = 50

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),]
)

dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"logs/fake")
writer_real = SummaryWriter(f"logs/real")
step = 0

for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        # where the second option of maximizing doesn't suffer from
        # saturating gradients
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1
        

In [ ]:
from pytorch3d.structures import Meshes

In [ ]:
#DCNN Implementation

In [ ]:
import torch 
import torch.nn as nn

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init()
        self.disc = nn.Sequential(
            #input: N x channels_img x 64 x 64
            nn.Conv2d(
                chennels_img, features_d, kernel_size=4, stride=2. padding=1),
             nn.LeakyRelU(0.2),
            self._block(features_d, features_d*2, 4, 2, 1), #16x16
            self._block(features_d*2, features_d*4, 4, 2, 1)#8x8
            self._block(features_d*4, features_d*8, 4, 2, 1), #4x4
             nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid()
        )
            
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
                ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
    def forward(self, x):
        return self.disc(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            #input Nx z_dim X1X1
            self._block(z_dim, features_d*16, 4, 1, 0),#N xf_g*16x4x4
            self._block(features_g*16, features_g*8, 4, 2, 1),#8x8
            self._block(features_g*8, features_g*4, 4, 2, 1),#16x16
            self._block(features_g*4, features_g*2, 4, 2, 1), #32x32
            nn.ConvTranspose2d(
                features_g*2, channels_img, kernel_size=4, stride=2. padding=1,
                nn.Tanh(), #[-1, 1]
        )
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),  
            nn.ReLU(),
        )
        def forward(self, x):
            return self.gen(x)
            
    def initialize_weights(model):
            for m in model.modules();
                if isinstance(m, (nn.Conv2d, nn.ConvTranspose2, nn.BatchNorm2d)):
                    nn.init.normal_(m.weight.data, 0.0, 0.02)
    def test():
            N, in_channels, H, W= 8, 3, 64, 64
            z_dim = 100
            x = torch.randn((N, in_channels, H, W))
            disc = Discriminator(in_channels, 8)
            initialize_wights(disc)
            assert disc(x).shape == (N, 1, 1, 1)
            gen = Generator(z_dim, in_channels, 8)
            initialize_weights(gen)
            z = torch. randn((N, z_dim, 1, 1))
            assert gen(z).shape == (N, in_channels, H, W)
            print('Success')

In [ ]:
#Training from model;
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from model import Discriminator, Generator, initilize_weights

In [ ]:
# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate= 5e-5
batch_size = 125
image_size = 64
channels_img = 1 #3
z_dim = 100 
num_epochs = 5
features_disc= 64
features_gen = 64
critic_iterations =5
wight_clip = 0.01

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transform.Normalize(
            [0.5 for _ in range(channels_img)], [0.5 for _ in range(channels_img)]),
    ]
)

In [ ]:
#root='celeb_dataset'
dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
gen = Generator(noise_dim, channels_img, features_gen).to(device)
critic = Discriminator(channels_img, features_disc).to(device)
initialize_weights(gen)
nitialize_weights(disc)

opt_gen = optim.RMSprop(gen.parameters(), lr=learning_rate, betas=(0.5, 0.999)) 
opt_disc = optim.RMSprop(critic.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [ ]:
fixed_noise = torch.rand(32, zdim, 1,1).to(device)

writer_real = SummaryWriter(f'logs/real')
writer_fake = SummaryWriter(f'logs/fake')

gen.train()
critic.train()

for epoch in range (num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        noise = torch.randn((batch_size, z_dim, 1, 1)).to(device)
        fake = gen(noise)
        for  - in range(critic_iterations):
            noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
            fake = gen(noise)
            critic_real = critic(real).reshape(-1)
            critic_fake = critic(fake).reshape(-1)
            loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake))
            loss.critic.backward(retain_graph=True)
            opt_critic.step()
            
            for p in critic.parameters
                p.data.clamp_(-weight_clip, weight_clip)
    output = critic(fake).reshape(-1)
    loss_gen = -torch.mean(output)
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()
        ##Train discriminator for max log(D((x))+ log(1 - D(G(z)))
        # disc_real = disc(real.reshape(-1)
        # loss_disc_real = criterion(disc_real, torchs.ones_like(disc_real))
        # disc_fake = disc(fake).reshape(-1)
        # loss_disc_fake = criterion(disc_fake, torchs.ones_like(disc_fake))
        # loss_disc = (loss_disc_real + loss_disc_fake) / 2
        # disc.zero_grad()
        # loss.disc.backward(retain_graph=True)
        # opt_disc.step()
        # ###Train Generator min log(1 -D*G(z))) <-->  max log(D(G(z))
        # output = disc(fake).reshape(-1)
        # loss_gen = criterion(output, torch.ones_like(output))
        # gen.zero_grad()
        # loss_gen.backward()
        # opt_gen.step()
    if batch_idx % 100 == 0:
        print(
            f'Epoch [{epoch}/{num_epochs}] Batch  {batch_idx} / {len(dataloader)} \
                Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
        )
        with torch.no_grad():
            fake = gen(fixed_noise)
                         #dropout (0>) 32 ex
            img_grid_real = torchvision.utils.make_grid(
                real[:32], normalize = True
            )
            img_grid_fake = torchvision.utils.make_grid(
                fake[:32], normalize = True
            )
                         
            writer_real.add_image('Real', img_grid_real, global_step=step)
            writer_fake.add_image('Fake', img_grid_fake, global_step=step)
        step += 1

In [ ]:
import torch 
import torch.nn as nn

In [ ]:
def gradient_penalty(critic, real, fake, device ='cpu'):
    batch_size, c, h, w = real.shape
    epsilon = torch.rand((batch_size, 1, 1, 1)).repeat(1, c, h, w).to(device)
    interpolated_images = real * epsilon + fake * (1 - epsilon)
    #critic score calculation
    mixed_scores = critic(interpolated_images)
    
    gradient = torch.autograd.grad(
        inputs=interpolated_images, 
        outpus=mized_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
        
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.L2norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm -1) ** 2)
    return gradient_penalty


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init()
        self.disc = nn.Sequential(
            #input: N x channels_img x 64 x 64
            nn.Conv2d(
                chennels_img, features_d, kernel_size=4, stride=2, padding=1),
             nn.LeakyRelU(0.2),
            self._block(features_d, features_d*2, 4, 2, 1), #16x16
            self._block(features_d*2, features_d*4, 4, 2, 1),#8x8
            self._block(features_d*4, features_d*8, 4, 2, 1), #4x4
             nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid()
        )
            
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
                ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )
    def forward(self, x):
        return self.disc(x)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from model import Discriminator, Generator, initilize_weights
from utils import gradient_penalty

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate= 1e-4
batch_size = 125
image_size = 64
channels_img = 1 #3
z_dim = 100 
num_epochs = 5
features_disc= 64
features_gen = 64
critic_iterations =5
lamda_gp = 10

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transform.Normalize(
            [0.5 for _ in range(channels_img)], [0.5 for _ in range(channels_img)]),
    ]
)

In [ ]:
dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
gen = Generator(noise_dim, channels_img, features_gen).to(device)
critic = Discriminator(channels_img, features_disc).to(device)
initialize_weights(gen)
nitialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=learning_rate, betas=(0.5, 0.9)) 
opt_disc = optim.Adam(critic.parameters(), lr=learning_rate, betas=(0.5, 0.9))

In [ ]:
fixed_noise = torch.rand(32, zdim, 1,1).to(device)

writer_real = SummaryWriter(f'logs/real')
writer_fake = SummaryWriter(f'logs/fake')

gen.train()
critic.train()

for epoch in range (num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        noise = torch.randn((batch_size, z_dim, 1, 1)).to(device)
        fake = gen(noise)
        for  - in range(critic_iterations):
            noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
            fake = gen(noise)
            critic_real = critic(real).reshape(-1)
            critic_fake = critic(fake).reshape(-1)
            gp = gradient_penalty(critic, real, fake, device=device)
            loss_critic = (
            -(torch.mean(critic_real) - torch.mean(critic_fake)) = lambda_gp*gp
            )
            loss.critic.backward(retain_graph=True)
            opt_critic.step()
            
            # for p in critic.parameters
            #     p.data.clamp_(-weight_clip, weight_clip)
    output = critic(fake).reshape(-1)
    loss_gen = -torch.mean(output)
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()
        ##Train discriminator for max log(D((x))+ log(1 - D(G(z)))
        # disc_real = disc(real.reshape(-1)
        # loss_disc_real = criterion(disc_real, torchs.ones_like(disc_real))
        # disc_fake = disc(fake).reshape(-1)
        # loss_disc_fake = criterion(disc_fake, torchs.ones_like(disc_fake))
        # loss_disc = (loss_disc_real + loss_disc_fake) / 2
        # disc.zero_grad()
        # loss.disc.backward(retain_graph=True)
        # opt_disc.step()
        # ###Train Generator min log(1 -D*G(z))) <-->  max log(D(G(z))
        # output = disc(fake).reshape(-1)
        # loss_gen = criterion(output, torch.ones_like(output))
        # gen.zero_grad()
        # loss_gen.backward()
        # opt_gen.step()
    if batch_idx % 100 == 0:
        print(
            f'Epoch [{epoch}/{num_epochs}] Batch  {batch_idx} / {len(dataloader)} \
                Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
        )
        with torch.no_grad():
            fake = gen(fixed_noise)
                         #dropout (0>) 32 ex
            img_grid_real = torchvision.utils.make_grid(
                real[:32], normalize = True
            )
            img_grid_fake = torchvision.utils.make_grid(
                fake[:32], normalize = True
            )
                         
            writer_real.add_image('Real', img_grid_real, global_step=step)
            writer_fake.add_image('Fake', img_grid_fake, global_step=step)
        step += 1